<a href="https://colab.research.google.com/github/Alex-Witt/DS-Unit-1-Sprint-4-Statistical-Tests-and-Experiments/blob/master/module3-introduction-to-bayesian-inference/LS_DS_143_Introduction_to_Bayesian_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lambda School Data Science Module 143

## Introduction to Bayesian Inference

!['Detector! What would the Bayesian statistician say if I asked him whether the--' [roll] 'I AM A NEUTRINO DETECTOR, NOT A LABYRINTH GUARD. SERIOUSLY, DID YOUR BRAIN FALL OUT?' [roll] '... yes.'](https://imgs.xkcd.com/comics/frequentists_vs_bayesians.png)

*[XKCD 1132](https://www.xkcd.com/1132/)*


## Prepare - Bayes' Theorem and the Bayesian mindset

Bayes' theorem possesses a near-mythical quality - a bit of math that somehow magically evaluates a situation. But this mythicalness has more to do with its reputation and advanced applications than the actual core of it - deriving it is actually remarkably straightforward.

### The Law of Total Probability

By definition, the total probability of all outcomes (events) if some variable (event space) $A$ is 1. That is:

$$P(A) = \sum_n P(A_n) = 1$$

The law of total probability takes this further, considering two variables ($A$ and $B$) and relating their marginal probabilities (their likelihoods considered independently, without reference to one another) and their conditional probabilities (their likelihoods considered jointly). A marginal probability is simply notated as e.g. $P(A)$, while a conditional probability is notated $P(A|B)$, which reads "probability of $A$ *given* $B$".

The law of total probability states:

$$P(A) = \sum_n P(A | B_n) P(B_n)$$

In words - the total probability of $A$ is equal to the sum of the conditional probability of $A$ on any given event $B_n$ times the probability of that event $B_n$, and summed over all possible events in $B$.

### The Law of Conditional Probability

What's the probability of something conditioned on something else? To determine this we have to go back to set theory and think about the intersection of sets:

The formula for actual calculation:

$$P(A|B) = \frac{P(A \cap B)}{P(B)}$$

![Visualization of set intersection](https://upload.wikimedia.org/wikipedia/commons/9/99/Venn0001.svg)

Think of the overall rectangle as the whole probability space, $A$ as the left circle, $B$ as the right circle, and their intersection as the red area. Try to visualize the ratio being described in the above formula, and how it is different from just the $P(A)$ (not conditioned on $B$).

We can see how this relates back to the law of total probability - multiply both sides by $P(B)$ and you get $P(A|B)P(B) = P(A \cap B)$ - replaced back into the law of total probability we get $P(A) = \sum_n P(A \cap B_n)$.

This may not seem like an improvement at first, but try to relate it back to the above picture - if you think of sets as physical objects, we're saying that the total probability of $A$ given $B$ is all the little pieces of it intersected with $B$, added together. The conditional probability is then just that again, but divided by the probability of $B$ itself happening in the first place.

### Bayes Theorem

Here is is, the seemingly magic tool:

$$P(A|B) = \frac{P(B|A)P(A)}{P(B)}$$

In words - the probability of $A$ conditioned on $B$ is the probability of $B$ conditioned on $A$, times the probability of $A$ and divided by the probability of $B$. These unconditioned probabilities are referred to as "prior beliefs", and the conditioned probabilities as "updated."

Why is this important? Scroll back up to the XKCD example - the Bayesian statistician draws a less absurd conclusion because their prior belief in the likelihood that the sun will go nova is extremely low. So, even when updated based on evidence from a detector that is $35/36 = 0.972$ accurate, the prior belief doesn't shift enough to change their overall opinion.

There's many examples of Bayes' theorem - one less absurd example is to apply to [breathalyzer tests](https://www.bayestheorem.net/breathalyzer-example/). You may think that a breathalyzer test that is 100% accurate for true positives (detecting somebody who is drunk) is pretty good, but what if it also has 8% false positives (indicating somebody is drunk when they're not)? And furthermore, the rate of drunk driving (and thus our prior belief)  is 1/1000.

What is the likelihood somebody really is drunk if they test positive? Some may guess it's 92% - the difference between the true positives and the false positives. But we have a prior belief of the background/true rate of drunk driving. Sounds like a job for Bayes' theorem!

$$
\begin{aligned}
P(Drunk | Positive) &= \frac{P(Positive | Drunk)P(Drunk)}{P(Positive)} \\
&= \frac{1 \times 0.001}{0.08} \\
&= 0.0125
\end{aligned}
$$

In other words, the likelihood that somebody is drunk given they tested positive with a breathalyzer in this situation is only 1.25% - probably much lower than you'd guess. This is why, in practice, it's important to have a repeated test to confirm (the probability of two false positives in a row is $0.08 * 0.08 = 0.0064$, much lower), and Bayes' theorem has been relevant in court cases where proper consideration of evidence was important.

## Derive Baye's Rule

\begin{align}
P(A|B) &= \frac{P(A \cap B)}{P(B)}\\
\Rightarrow P(A|B)P(B) &= P(A \cap B)\\
P(B|A) &= \frac{P(B \cap A)}{P(A)}\\
\Rightarrow P(B|A)P(A) &= P(B \cap A)\\
\Rightarrow P(A|B)P(B) &= P(B|A)P(A) \\
P(A \cap B) &= P(B \cap A)\\
P(A|B) &= \frac{P(B|A) \times P(A)}{P(B)}
\end{align}

## Live Lecture - Deriving Bayes' Theorem, Calculating Bayesian Confidence

Notice that $P(A|B)$ appears in the above laws - in Bayesian terms, this is the belief in $A$ updated for the evidence $B$. So all we need to do is solve for this term to derive Bayes' theorem. Let's do it together!

In [0]:
# Activity 2 - Use SciPy to calculate Bayesian confidence intervals
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.bayes_mvs.html#scipy.stats.bayes_mvs

In [5]:
from scipy import stats
import numpy as np


np.random.seed(seed=42)
coinflips = np.random.binomial(n=1, p=.5, size = 100)

print(coinflips)


[0 1 1 1 0 0 0 1 1 1 0 1 1 0 0 0 0 1 0 0 1 0 0 0 0 1 0 1 1 0 1 0 0 1 1 1 0
 0 1 0 0 0 0 1 0 1 0 1 1 0 1 1 1 1 1 1 0 0 0 0 0 0 1 0 0 1 0 1 0 1 1 0 0 1
 1 1 1 0 0 0 1 1 0 0 0 0 1 1 1 0 0 1 1 1 1 0 1 0 0 0]


In [0]:
def confidence_interval(data, confidence =.95):
  n = len(data)
  mean = sum(data)/n
  data = np.array(data)
  stderr = stats.sem(data)
  interval = stderr * stats.t.ppf((1 + confidence) /2.0, n-1)
  return(mean, mean-interval, mean+interval)
  
  


In [7]:
confidence_interval(coinflips)

(0.47, 0.3704689875017368, 0.5695310124982632)

In [8]:
help(stats.bayes_mvs)

Help on function bayes_mvs in module scipy.stats.morestats:

bayes_mvs(data, alpha=0.9)
    Bayesian confidence intervals for the mean, var, and std.
    
    Parameters
    ----------
    data : array_like
        Input data, if multi-dimensional it is flattened to 1-D by `bayes_mvs`.
        Requires 2 or more data points.
    alpha : float, optional
        Probability that the returned confidence interval contains
        the true parameter.
    
    Returns
    -------
    mean_cntr, var_cntr, std_cntr : tuple
        The three results are for the mean, variance and standard deviation,
        respectively.  Each result is a tuple of the form::
    
            (center, (lower, upper))
    
        with `center` the mean of the conditional pdf of the value given the
        data, and `(lower, upper)` a confidence interval, centered on the
        median, containing the estimate to a probability ``alpha``.
    
    See Also
    --------
    mvsdist
    
    Notes
    -----
    Each

In [9]:
stats.bayes_mvs(coinflips, alpha = .95)

(Mean(statistic=0.47, minmax=(0.37046898750173674, 0.5695310124982632)),
 Variance(statistic=0.25680412371134015, minmax=(0.1939698977025208, 0.3395533426586547)),
 Std_dev(statistic=0.5054540733507159, minmax=(0.44042013771229943, 0.5827120581030176)))

In [10]:
coinflips_mean_dist, _, _ = stats.mvsdist(coinflips)
coinflips_mean_dist

In [11]:
help(stats._distn_infrastructure.rv_frozen)

Help on class rv_frozen in module scipy.stats._distn_infrastructure:

class rv_frozen(builtins.object)
 |  # Frozen RV class
 |  
 |  Methods defined here:
 |  
 |  __init__(self, dist, *args, **kwds)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  cdf(self, x)
 |  
 |  entropy(self)
 |  
 |  expect(self, func=None, lb=None, ub=None, conditional=False, **kwds)
 |  
 |  interval(self, alpha)
 |  
 |  isf(self, q)
 |  
 |  logcdf(self, x)
 |  
 |  logpdf(self, x)
 |  
 |  logpmf(self, k)
 |  
 |  logsf(self, x)
 |  
 |  mean(self)
 |  
 |  median(self)
 |  
 |  moment(self, n)
 |  
 |  pdf(self, x)
 |  
 |  pmf(self, k)
 |  
 |  ppf(self, q)
 |  
 |  rvs(self, size=None, random_state=None)
 |  
 |  sf(self, x)
 |  
 |  stats(self, moments='mv')
 |  
 |  std(self)
 |  
 |  var(self)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (i

In [12]:
#Bootstrapping new data from existing data!

coinflips_mean_dist.rvs(100)

array([0.47447628, 0.51541425, 0.54722018, 0.4589882 , 0.51501386,
       0.53819192, 0.43382292, 0.53546659, 0.47026173, 0.44967562,
       0.4621107 , 0.42691904, 0.37324325, 0.47531437, 0.46052277,
       0.48711257, 0.52456771, 0.43332181, 0.49545882, 0.44671454,
       0.47520117, 0.47047251, 0.41828918, 0.50159477, 0.42965501,
       0.45273383, 0.48045849, 0.45342529, 0.48238344, 0.53966291,
       0.48230241, 0.48073422, 0.48553525, 0.47962228, 0.41274185,
       0.42892633, 0.5170948 , 0.42678096, 0.42249309, 0.51499109,
       0.47059199, 0.39903942, 0.41790336, 0.46406817, 0.42232382,
       0.42163269, 0.47848227, 0.48232842, 0.4731858 , 0.51077244,
       0.3957508 , 0.48504646, 0.49014295, 0.53252732, 0.45495376,
       0.47883978, 0.60393033, 0.4492549 , 0.44797902, 0.54782121,
       0.43380002, 0.5760073 , 0.36941266, 0.44467418, 0.4939245 ,
       0.45278835, 0.55635162, 0.48695459, 0.39080983, 0.45948606,
       0.2941779 , 0.35950718, 0.44805696, 0.4725126 , 0.42218

## Assignment - Code it up!

Most of the above was pure math - now write Python code to reproduce the results! This is purposefully open ended - you'll have to think about how you should represent probabilities and events. You can and should look things up, and as a stretch goal - refactor your code into helpful reusable functions!

Specific goals/targets:

1. Write a function `def prob_drunk_given_positive(prob_drunk_prior, prob_positive, prob_positive_drunk)` that reproduces the example from lecture, and use it to calculate and visualize a range of situations
2. Explore `scipy.stats.bayes_mvs` - read its documentation, and experiment with it on data you've tested in other ways earlier this week
3. Create a visualization comparing the results of a Bayesian approach to a traditional/frequentist approach
4. In your own words, summarize the difference between Bayesian and Frequentist statistics

If you're unsure where to start, check out [this blog post of Bayes theorem with Python](https://dataconomy.com/2015/02/introduction-to-bayes-theorem-with-python/) - you could and should create something similar!

Stretch goals:

- Apply a Bayesian technique to a problem you previously worked (in an assignment or project work) on from a frequentist (standard) perspective
- Check out [PyMC3](https://docs.pymc.io/) (note this goes beyond hypothesis tests into modeling) - read the guides and work through some examples
- Take PyMC3 further - see if you can build something with it!

### Imports

In [0]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from scipy import stats

### Drunk Driving

## Comparison to Other Work

I'm going to pull my work from yesterday's assignment. I'm going to compare the confidence intervals of both parties, and plot them back to back.

### Data Management

In [14]:
df = pd.read_csv('https://www.dropbox.com/s/eykydmqlt003trd/house-votes-84.data?dl=1', 
                 na_values = ['?'] , 
                 header = None,
                 names = ['Party','handicapped-infants','water',
                          'budget','health_costs',
                          'el_salvador','religious_groups_in_school','anti_satellite_wep',
                          'contra','mx_missile','immigration',
                          'synfuel','education',
                          'superfund','crime','duty_free_export',
                          'south_africa'])

df = df.replace({'n': 0 , 'y': 1})

dem = df.loc[df['Party'] == 'democrat']
rep = df.loc[df['Party'] == 'republican']

df.head()

,Party,handicapped-infants,water,budget,health_costs,el_salvador,religious_groups_in_school,anti_satellite_wep,contra,mx_missile,immigration,synfuel,education,superfund,crime,duty_free_export,south_africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


In [0]:
#I'm going to establish some lists and variables to use later. 

bills = ['handicapped-infants', 'water', 'budget', 'health_costs',
       'el_salvador', 'religious_groups_in_school', 'anti_satellite_wep',
       'contra', 'mx_missile', 'immigration', 'synfuel', 'education',
       'superfund', 'crime', 'duty_free_export', 'south_africa']
parties = ['republicans','democrats']

upper_hypothesis = 80    # Upper Null Hypothesis for Bills a party supports 
lower_hypothesis = 20    # Lower Null Hypothesis for Bills a Party opposes


In [0]:
def confidence_interval(data, confidence=.95):
  n = len(data)
  mean = sum(data)/n
  data = np.array(data)
  stderr = stats.sem(data)
  interval = stderr * stats.t.ppf((1 + confidence) / 2.0, n-1)
  return (mean , mean-interval, mean+interval)

def f_conf (party, bill):    # This function to call a function just adds the dropna
  r = confidence_interval(party[bill].dropna())   
  return r      

def b_conf (party,bill):
  b = stats.bayes_mvs(party[bill].dropna(), alpha = .95)
  return b


In [0]:
## This is my frequentist Dataframe


r_upper = []        # Creating a bunch of lists to append
r_lower = []        # These will be used as my columns in a new Dataframe
r_mean = []

d_upper = []
d_lower = []
d_mean = []


for bill in bills:
                  #I also want to bring the values up to percentages. It looks nicer. 
  r_upper.append(f_conf(rep,bill)[2]*100)
  r_lower.append(f_conf(rep,bill)[1]*100)
  r_mean.append(f_conf(rep,bill)[0]*100)
    
  d_upper.append(f_conf(dem,bill)[2]*100)
  d_lower.append(f_conf(dem,bill)[1]*100)
  d_mean.append(f_conf(dem,bill)[0]*100)


    ### New Dataframe with Confidence Intervals 

vc = pd.DataFrame({"Bill": bills, 'R Upper Limit': r_upper, 'R Lower Limit': r_lower, 
                   'Republican Support': r_mean, 'D Upper Limit': d_upper, 'D Lower Limit': d_lower, 
                   'Democrat Support': d_mean, "Null Hypothesis Rejected": True})  
    
  # I also want to create a Null Hypothesis Rejected column to be appended below. 

names = []
       ##This list is setting up my condition for determining if the null hypothesis is rejected. 
names.append(vc[(upper_hypothesis <= vc['D Upper Limit']) &
                (vc['D Lower Limit'] <= upper_hypothesis)].Bill.tolist())
names.append((vc[(lower_hypothesis <= vc['D Upper Limit']) &
                 (vc['D Lower Limit'] <= lower_hypothesis)]).Bill.tolist())
names.append((vc[(upper_hypothesis <= vc['R Upper Limit']) &
                 (vc['R Lower Limit'] <= upper_hypothesis)]).Bill.tolist())
names.append((vc[(lower_hypothesis <= vc['R Upper Limit']) &
                 (vc['R Lower Limit'] <= lower_hypothesis)]).Bill.tolist())

flat_list = []  ## The above method creates a list of lists. This flattens the list
for sublist in names:
    for item in sublist:
        flat_list.append(item)
        
names = list(set(flat_list))   ##This returns only unique values in my list. 

   ## Now to replace True values with False where the Null Hypothesis is not rejected. 
vc.loc[vc['Bill'].isin(names), "Null Hypothesis Rejected"] = False  



In [0]:
#This is my Baysian Dataframe

r_upper = []        # Creating a bunch of lists to append
r_lower = []        # These will be used as my columns in a new Dataframe
r_mean = []

d_upper = []
d_lower = []
d_mean = []


for bill in bills:
                  #I also want to bring the values up to percentages. It looks nicer. 
  r_upper.append(b_conf(rep,bill)[0][1][1]*100)
  r_lower.append(b_conf(rep,bill)[0][1][0]*100)
  r_mean.append(b_conf(rep,bill)[0][0]*100)
    
  d_upper.append(b_conf(dem,bill)[0][1][1]*100)
  d_lower.append(b_conf(dem,bill)[0][1][0]*100)
  d_mean.append(b_conf(dem,bill)[0][0]*100)


    ### New Dataframe with Confidence Intervals 

bays = pd.DataFrame({"Bill": bills, 'R Upper Limit': r_upper, 'R Lower Limit': r_lower, 
                   'Republican Support': r_mean, 'D Upper Limit': d_upper, 'D Lower Limit': d_lower, 
                   'Democrat Support': d_mean, "Null Hypothesis Rejected": True})  
    
  # I also want to create a Null Hypothesis Rejected column to be appended below. 

names = []
       ##This list is setting up my condition for determining if the null hypothesis is rejected. 
names.append(bays[(upper_hypothesis <= bays['D Upper Limit']) &
                (vc['D Lower Limit'] <= upper_hypothesis)].Bill.tolist())
names.append((bays[(lower_hypothesis <= bays['D Upper Limit']) &
                 (bays['D Lower Limit'] <= lower_hypothesis)]).Bill.tolist())
names.append((bays[(upper_hypothesis <= bays['R Upper Limit']) &
                 (bays['R Lower Limit'] <= upper_hypothesis)]).Bill.tolist())
names.append((bays[(lower_hypothesis <= bays['R Upper Limit']) &
                 (bays['R Lower Limit'] <= lower_hypothesis)]).Bill.tolist())

flat_list = []  ## The above method creates a list of lists. This flattens the list
for sublist in names:
    for item in sublist:
        flat_list.append(item)
        
names = list(set(flat_list))   ##This returns only unique values in my list. 

   ## Now to replace True values with False where the Null Hypothesis is not rejected. 
bays.loc[vc['Bill'].isin(names), "Null Hypothesis Rejected"] = False  



### Let's check the results:

In [19]:
vc

,Bill,D Lower Limit,D Upper Limit,Democrat Support,Null Hypothesis Rejected,R Lower Limit,R Upper Limit,Republican Support
0,handicapped-infants,54.459270,66.470963,60.465116,False,12.765166,24.810591,18.787879
1,water,43.824514,56.593896,50.209205,True,42.526571,58.824780,50.675676
2,budget,84.994352,92.697956,88.846154,True,8.143520,18.685748,13.414634
3,health_costs,2.633186,8.177625,5.405405,True,97.100674,100.475083,98.787879
4,el_salvador,16.486311,26.650944,21.568627,False,91.839795,98.463236,95.151515
5,religious_groups_in_school,41.539183,53.809655,47.674419,True,85.098749,94.419324,89.759036
6,anti_satellite_wep,72.078202,82.361952,77.220077,False,17.420089,30.728059,24.074074
7,contra,78.308452,87.471016,82.889734,False,9.595477,20.977771,15.286624
8,mx_missile,70.439401,81.173502,75.806452,False,6.593486,16.436817,11.515152
9,immigration,41.075744,53.220834,47.148289,True,48.099596,63.415556,55.757576


In [20]:
bays

,Bill,D Lower Limit,D Upper Limit,Democrat Support,Null Hypothesis Rejected,R Lower Limit,R Upper Limit,Republican Support
0,handicapped-infants,54.459270,66.470963,60.465116,False,12.765166,24.810591,18.787879
1,water,43.824514,56.593896,50.209205,True,42.526571,58.824780,50.675676
2,budget,84.994352,92.697956,88.846154,True,8.143520,18.685748,13.414634
3,health_costs,2.633186,8.177625,5.405405,True,97.100674,100.475083,98.787879
4,el_salvador,16.486311,26.650944,21.568627,False,91.839795,98.463236,95.151515
5,religious_groups_in_school,41.539183,53.809655,47.674419,True,85.098749,94.419324,89.759036
6,anti_satellite_wep,72.078202,82.361952,77.220077,False,17.420089,30.728059,24.074074
7,contra,78.308452,87.471016,82.889734,False,9.595477,20.977771,15.286624
8,mx_missile,70.439401,81.173502,75.806452,False,6.593486,16.436817,11.515152
9,immigration,41.075744,53.220834,47.148289,True,48.099596,63.415556,55.757576


In [25]:
print('frequentis mean and confidence interval:' ,f_conf(rep,'water'))
print()
print('Baysian mean and confidence interval:')
b_conf(rep,'water')[0]

frequentis mean and confidence interval: (0.5067567567567568, 0.42526571045979167, 0.5882478030537219)

Baysian mean and confidence interval:


Mean(statistic=0.5067567567567568, minmax=(0.42526571045979167, 0.5882478030537219))

In [30]:
print("Frequentist Standard Deviation:" , rep['water'].std())
print()
print("Baysian Standard Deviation:" , b_conf(rep, 'water')[2][0])

Frequentist Standard Deviation: 0.5016519868797793

Baysian Standard Deviation: 0.5042297029005319


Looks like the confidence interval is the same between Frequentist and Baysian methods. However, the Standard deviation is slightly different. In the above example, the differnce between the two is roughly 0.1%. Not an enormous divergence and would only slightly affect the calculation of Standard Error and various T/chi tests.

## Resources

- [Worked example of Bayes rule calculation](https://en.wikipedia.org/wiki/Bayes'_theorem#Examples) (helpful as it fully breaks out the denominator)
- [Source code for mvsdist in scipy](https://github.com/scipy/scipy/blob/90534919e139d2a81c24bf08341734ff41a3db12/scipy/stats/morestats.py#L139)